# XGBoost solution

**_______________________________________________________________________________________________________________________________**

**Загрузим все необходимые зависимости и сущности, модули:**

In [1]:
%run ../preset.py

---------------------------------------------------------------------------------------------------------------------------

**Загрузим данные:**

In [2]:
df = pd.read_parquet(f"../" + prep_data_url)

___________________________________________________________________________________________________________________________

**Обзор dataframe-a:**

In [3]:
df.head(3)

,subject_name,actual_consumption,datetime
index,,,
2011-01-01 00:00:00,Алтайский край,1313.0,2011-01-01 00:00:00
2011-01-01 00:00:00,Амурская область,870.0,2011-01-01 00:00:00
2011-01-01 00:00:00,Архангельская область,970.0,2011-01-01 00:00:00


**Рассмотрим информацию о dataframe-е:**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8478449 entries, 2011-01-01 00:00:00 to 2024-07-17 00:00:00
Data columns (total 3 columns):
 #   Column              Dtype  
---  ------              -----  
 0   subject_name        object 
 1   actual_consumption  float64
 2   datetime            object 
dtypes: float64(1), object(2)
memory usage: 258.7+ MB


___________________________________________________________________________________________________________________________

**Создадим признаки для обучения модели:**

In [5]:
df["year"] = df["datetime"].apply(get_year)

In [6]:
df["month"] = df["datetime"].apply(get_month)

In [7]:
df["day_of_month"] = df["datetime"].apply(get_day_month)

In [8]:
df["hour"] = df["datetime"].apply(get_hour)

In [9]:
df["day_of_week"] = df["datetime"].apply(get_day_week)

In [10]:
df["day_of_year"] = df["datetime"].apply(get_day_year)

In [11]:
df["week_of_year"] = df["datetime"].apply(get_week_year)

In [12]:
df["quarter"] = df["datetime"].apply(get_quarter)

In [13]:
df["holiday"] = df["datetime"].apply(is_holiday)

___________________________________________________________________________________________________________________________

**Рассмотрим dataframe с новыми признаками:**

In [14]:
df.head(3)

,subject_name,actual_consumption,datetime,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday
index,,,,,,,,,,,,
2011-01-01 00:00:00,Алтайский край,1313.0,2011-01-01 00:00:00,2011,1,1,0,6,1,52,3,1
2011-01-01 00:00:00,Амурская область,870.0,2011-01-01 00:00:00,2011,1,1,0,6,1,52,3,1
2011-01-01 00:00:00,Архангельская область,970.0,2011-01-01 00:00:00,2011,1,1,0,6,1,52,3,1


In [15]:
df.tail(3)

,subject_name,actual_consumption,datetime,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday
index,,,,,,,,,,,,
2024-07-17 00:00:00,Чувашская Республика - Чувашия,486.0,2024-07-17 00:00:00,2024,7,17,0,3,199,29,3,0
2024-07-17 00:00:00,Южно-Якутский энергорайон,358.0,2024-07-17 00:00:00,2024,7,17,0,3,199,29,3,0
2024-07-17 00:00:00,Ярославская область,766.0,2024-07-17 00:00:00,2024,7,17,0,3,199,29,3,0


___________________________________________________________________________________________________________________________

**Очистим данные от выходных, так как учитываются только рабочие дни:**

In [16]:
df = df[df["day_of_week"] < 6]

**Рассмотрим подчищенный dataframe:**

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6057270 entries, 2011-01-03 00:00:00 to 2024-07-17 00:00:00
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   subject_name        object 
 1   actual_consumption  float64
 2   datetime            object 
 3   year                int64  
 4   month               int64  
 5   day_of_month        int64  
 6   hour                int64  
 7   day_of_week         int64  
 8   day_of_year         int64  
 9   week_of_year        int64  
 10  quarter             int64  
 11  holiday             int64  
dtypes: float64(1), int64(9), object(2)
memory usage: 600.8+ MB


___________________________________________________________________________________________________________________________

**Рассмотрим подробнее частоту встречаемости значений признаков:**

In [18]:
df["year"].value_counts().tail(3)

2011    386706
2013    384480
2024    255513
Name: year, dtype: int64

In [19]:
df["month"].value_counts().tail(3)

11    465316
2     464193
12    451748
Name: month, dtype: int64

In [20]:
df["day_of_month"].value_counts().tail(3)

29    187649
30    180709
31    103940
Name: day_of_month, dtype: int64

In [21]:
df["hour"].value_counts().tail(3)

23    252086
14    252078
22    251842
Name: hour, dtype: int64

In [22]:
df["day_of_week"].value_counts().tail(3)

2    1211737
5    1210970
4    1210531
Name: day_of_week, dtype: int64

In [23]:
df["day_of_year"].value_counts().tail(3)

358    12216
365     3918
366      147
Name: day_of_year, dtype: int64

In [24]:
df["week_of_year"].value_counts().tail(3)

50    105248
52    103804
53     14162
Name: week_of_year, dtype: int64

In [25]:
df["quarter"].value_counts()

3    6057270
Name: quarter, dtype: int64

In [26]:
df["holiday"].value_counts()

0    5814001
1     243269
Name: holiday, dtype: int64

___________________________________________________________________________________________________________________________

**Удалим ненужный признак `datetime`:**

In [27]:
df.drop(columns=["datetime"], inplace=True)

**Обновлённый dataframe:**

In [28]:
df.head(3)

,subject_name,actual_consumption,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday
index,,,,,,,,,,,
2011-01-03 00:00:00,Алтайский край,1230.0,2011,1,3,0,1,3,1,3,1
2011-01-03 00:00:00,Амурская область,995.0,2011,1,3,0,1,3,1,3,1
2011-01-03 00:00:00,Архангельская область,923.0,2011,1,3,0,1,3,1,3,1


___________________________________________________________________________________________________________________________

**Рассмотрим подробную информацию о dataframe-е:**

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6057270 entries, 2011-01-03 00:00:00 to 2024-07-17 00:00:00
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   subject_name        object 
 1   actual_consumption  float64
 2   year                int64  
 3   month               int64  
 4   day_of_month        int64  
 5   hour                int64  
 6   day_of_week         int64  
 7   day_of_year         int64  
 8   week_of_year        int64  
 9   quarter             int64  
 10  holiday             int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 554.6+ MB


In [30]:
df.describe()

,actual_consumption,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday
count,6.057270e+06,6.057270e+06,6.057270e+06,6.057270e+06,6.057270e+06,6.057270e+06,6.057270e+06,6.057270e+06,6057270.0,6.057270e+06
mean,1.644817e+03,2.017465e+03,6.392757e+00,1.565209e+01,1.149606e+01,2.999541e+00,1.790753e+02,2.609543e+01,3.0,4.016149e-02
std,2.148254e+03,3.880011e+00,3.395319e+00,8.793401e+00,6.922168e+00,1.414134e+00,1.037535e+02,1.481422e+01,0.0,1.963379e-01
min,4.000000e+00,2.011000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.0,0.000000e+00
25%,4.770000e+02,2.014000e+03,3.000000e+00,8.000000e+00,5.000000e+00,2.000000e+00,9.000000e+01,1.300000e+01,3.0,0.000000e+00
50%,9.510000e+02,2.018000e+03,6.000000e+00,1.600000e+01,1.100000e+01,3.000000e+00,1.770000e+02,2.600000e+01,3.0,0.000000e+00
75%,1.795000e+03,2.021000e+03,9.000000e+00,2.300000e+01,1.700000e+01,4.000000e+00,2.680000e+02,3.900000e+01,3.0,0.000000e+00
max,1.984600e+04,2.024000e+03,1.200000e+01,3.100000e+01,2.300000e+01,5.000000e+00,3.660000e+02,5.300000e+01,3.0,1.000000e+00


___________________________________________________________________________________________________________________________

**Преобразуем название субъекта, как категориальный признак:**

In [31]:
cat_subj = pd.get_dummies(df["subject_name"])

**Рассмотрим преобразованные данные:**

In [32]:
cat_subj.head()

,Алтайский край,Амурская область,Архангельская область,Астраханская область,Белгородская область,Брянская область,Владимирская область,Волгоградская область,Вологодская область,Воронежская область,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2011-01-03 00:00:00,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___________________________________________________________________________________________________________________________

**Удалим старый признак и присоединим новый:**

In [33]:
df.drop(columns=["subject_name"], inplace=True)

In [34]:
df = pd.concat([df, cat_subj], axis=1)

**Рассмотрим dataframe с обновлённым признаком:**

In [35]:
df.head(3)

,actual_consumption,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2011-01-03 00:00:00,1230.0,2011,1,3,0,1,3,1,3,1,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,995.0,2011,1,3,0,1,3,1,3,1,...,0,0,0,0,0,0,0,0,0,0
2011-01-03 00:00:00,923.0,2011,1,3,0,1,3,1,3,1,...,0,0,0,0,0,0,0,0,0,0


___________________________________________________________________________________________________________________________

**Сохраним готовый датасет:**

In [36]:
df.to_parquet("../../prepare_data/prep_df.gzip")

___________________________________________________________________________________________________________________________

**Отберём целевой признак:**

In [37]:
df = pd.read_parquet("../../prepare_data/feature_data.gzip")

In [38]:
y = df["actual_consumption"]

**Рассмотрим целевой признак:**

In [39]:
y.head()

index
2012-02-20 00:00:00    1302.0
2012-02-20 00:00:00    1091.0
2012-02-20 00:00:00     946.0
2012-02-20 00:00:00     615.0
2012-02-20 00:00:00    1679.0
Name: actual_consumption, dtype: float64

___________________________________________________________________________________________________________________________

**Отберём переменные-регрессоры:**

In [40]:
x = df.drop(columns=["actual_consumption"])

**Рассмотрим отобранные признаки:**

In [62]:
x.head(3)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 00:00:00,115803,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,1,0,0,0,0
2024-07-17 00:00:00,115797,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,0,1,0,0,0
2024-07-17 00:00:00,115806,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,0,0,1,0,0


In [63]:
y

index
2012-02-20 00:00:00    1302.0
2012-02-20 00:00:00    1091.0
2012-02-20 00:00:00     946.0
2012-02-20 00:00:00     615.0
2012-02-20 00:00:00    1679.0
                        ...  
2024-07-17 00:00:00    3854.0
2024-07-17 00:00:00     412.0
2024-07-17 00:00:00     486.0
2024-07-17 00:00:00     358.0
2024-07-17 00:00:00     766.0
Name: actual_consumption, Length: 5588845, dtype: float64

___________________________________________________________________________________________________________________________

**Разобъём данные на три выборки (тренировочную, тестовую, валидационную) без перемешивания:**

In [42]:
x_train, x_other, y_train, y_other = train_test_split(
                                                      x,  # Значения признаков
                                                      y,  # Значения целевой переменной
                                                      test_size=0.3,  # Размер тестовой выборки
                                                      shuffle=False  # Отключаем перемешивание данных
                                                     )

In [43]:
x_test, x_val, y_test, y_val = train_test_split(
                                                x_other,  # Значения признаков
                                                y_other,  # Значения целевой переменной
                                                test_size=0.3,  # Размер тестовой выборки
                                                shuffle=False  # Отключаем перемешивание данных
                                               )

**Рассмотрим полученные выборки:**

In [44]:
x_train

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84522,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84522,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
x_test

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,34687,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84522,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-20 10:00:00,106428,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,106426,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,106430,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
x_val

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2023-06-20 10:00:00,106422,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,106419,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,106426,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,106426,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-06-20 10:00:00,56590,2023,6,20,10,2,171,25,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 00:00:00,115803,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,1,0,0,0,0
2024-07-17 00:00:00,115797,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,0,1,0,0,0
2024-07-17 00:00:00,115806,2024,7,17,0,3,199,29,3,0,...,0,0,0,0,0,0,0,1,0,0


___________________________________________________________________________________________________________________________

**Ознакомимся с моделью машинного обучения:**

In [47]:
help(XGBRegressor)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  See :doc:`/python/sklearn_estimator` for more information.
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : Optional[int]
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          d

___________________________________________________________________________________________________________________________

**Создадим модель машинного обучения:**

In [48]:
main_model = xgb.XGBRegressor()

___________________________________________________________________________________________________________________________

**Рассмотрим параметры сетки:**

In [49]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "score_samples", "predict", "predict_proba",
 |  "decision_function", "transform" and "inverse_transform" if they are
 |  implemented in the estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either est

**Установим параметры сетки:**

In [50]:
# parameters = {
# #               "max_depth":[2, 3],
# #               "max_leaves": [4, 5],
# #               "learning_rate":[0.05, 0.025],
#               "n_estimators": [1000]
#              }

**Создадим сетку:**

In [51]:
# GS_model = GridSearchCV(
#                         main_model,
#                         param_grid=parameters,
#                         cv=4,
#                         scoring="neg_mean_squared_error"
#                        )

**Найдём лучшие параметры при помощи сетки:**

In [52]:
# GS_model.fit(x_test, y_test)

In [64]:
main_model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

**Рассмотрим лучшие параметры подобранные с помощью сетки:**

In [54]:
# print(GS_model.best_params_)

___________________________________________________________________________________________________________________________

**Спрогнозируем данные на валидационной выборке:**

In [65]:
y_pred = main_model.predict(x_test)

**Оценим полученный результат:**

In [66]:
check_res(y_pred, y_test)

Значения метрик:
                              MSE - 10129.747061818485
                              MAE - 38.16364790175351
                              MAPE - 0.026539338355456445


In [67]:
y_pred_val = main_model.predict(x_val)

In [68]:
check_res(y_pred_val, y_val)

Значения метрик:
                              MSE - 22645.15184674116
                              MAE - 44.3958328025134
                              MAPE - 0.02725785732804807
